In [2]:
import numpy as np
import pandas as pd
import darr
from birdwatcher import VideoFileStream, CoordinateArrays
from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool, ZoomInTool
import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage
output_notebook()

Loading BokehJS ...

In [3]:
coordinates = darr.RaggedArray('./zf20s_low_movementknn_me2/coordinates.drarr/') 
coordsmean = darr.Array('./zf20s_low_movementknn_me2/coordsmean.darr/')
coordscount = darr.Array('./zf20s_low_movementknn_me2/coordscount.darr/')


OSError: 'zf20s_low_movementknn_me2/coordinates.drarr' does not exist

In [4]:
def plot_meancoord(coordsmean, coordscount, countthreshold=0, decimate=None):
    
    if decimate is None:
        x, y = coordsmean[:,0], coordsmean[:,1]
        c = coordscount[:]
        decimate=1
    else:
        x, y = coordsmean[::decimate,0], coordsmean[::decimate,1]
        c = coordscount[::decimate]
    thi = c > countthreshold
    desc = np.arange(len(x))*decimate
    source = ColumnDataSource(
            data=dict(
                x=x[thi],
                y=y[thi],
                desc=desc[thi],
            )
        )

    hover = HoverTool(
            tooltips=[
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("desc", "@desc"),
            ]
    )

    p = figure(title="coordinate mean", x_axis_label='x', y_axis_label='y', width=900, height=800, 
               tools=[hover, ZoomInTool()])
    p.scatter('x', 'y', legend="mean coordinate", source=source, size=3)
    show(p)

In [5]:
plot_meancoord(coordsmean, coordscount, decimate=None)

In [6]:
df = pd.DataFrame({'x_col': coordsmean[:,0], 'y_col': coordsmean[:,1] })

In [8]:
def base_plot():
    p = figure(
        x_range=(0, 1920),
        y_range=(0, 1080),
        tools='pan,wheel_zoom,box_zoom,reset', 
        plot_width=900, 
        plot_height=800,
    )
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.axis_label = "Distance, left-right"
    p.yaxis.axis_label = "Distance, top-bottom"
    p.xaxis.axis_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    return p
    
options = dict(line_color=None, fill_color='blue', size=5)

In [9]:
cvs = ds.Canvas(plot_width=900, plot_height=800)
agg = cvs.points(df, 'x_col', 'y_col', ds.count())
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='log')
p = base_plot()
pipeline = ds.Pipeline(df, ds.Point('x_col', 'y_col'))
InteractiveImage(p, pipeline)